In [ ]:
def remove_firstline(oldfile) :
    with open(oldfile,'r') as f:
        newfile = oldfile + '.1'
        print(newfile)
        with open(newfile,'a') as f1:
            f.next() # skip header line
            for line in f:
                f1.write(line)
            f1.close()

In [ ]:
### handling the stock/index csv data
import glob, os

def handle_index_data() :
    os.chdir("/home/jianbo/Data/index_data")
    for file in glob.glob("*.csv"):
        print(file)
        remove_firstline(file)
        os.remove(file)

def handle_stock_data() :
    os.chdir("/home/jianbo/Data/stock_data")
    for file in glob.glob("*.csv"):
        print(file)
        remove_firstline(file)
        os.remove(file)

# Import historical price data into database

In [1]:
import csv
import MySQLdb
import glob, os
import pymongo

def get_mysql_conn() :
    # init DB
    mydb = MySQLdb.connect(host='localhost',
        user='root',
        passwd='eth@2002',
        db='darwin')
    return mydb

def get_mongo_conn() :
    try :
        mgdb = pymongo.MongoClient('localhost', 27017)
        print 'success'
    except :
        print 'failed'
    return mgdb
    

In [2]:
mgdb = get_mongo_conn()

success


In [3]:
def load_dailyprice_mysql() :
    # init DB
    mydb = get_mysql_conn()
    cursor = mydb.cursor()

    # change the directory to stock data
    os.chdir('/home/jianbo/Data/stock_data')
    for stock_file in glob.glob('*.csv') :
        csv_data = csv.reader(file(stock_file))
        print "loading file " + stock_file
        count = 0

        for row in csv_data:
            if count == 0:
                count = count + 1
                continue
            try:
                cursor.execute('INSERT INTO Stock_Price_Daily(code,date,open,high,low,close, price_change,volume,money,traded_market_value,market_value, turnover,adjust_price,report_type,report_date,PE_TTM,PS_TTM,PC_TTM,PB) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)', row)
                mydb.commit()
            except:
                mydb.rollback()
        os.remove(stock_file)


    # close the connection
    mydb.close()
                       

# Import Stock Fundamental Information